This notebook contains scripts used to generate 64x64 images usable by the U-Net model from 512x512 images from the GalaxyZoo3D project.


In [127]:
IMAGE_PATH = "E:\Documents\Thesis\spiral-galaxies\data\galaxyzoo3d\orig_img"
MASK_PATH = "E:\Documents\Thesis\spiral-galaxies\data\galaxyzoo3d\orig_msk"
BINARY_MASK_OUT_PATH = (
    "E:\Documents\Thesis\spiral-galaxies\data\galaxyzoo3d\\bin_msk"
)
RESCALED_IMAGE_OUT_PATH = (
    "E:\Documents\Thesis\spiral-galaxies\data\galaxyzoo3d\\rescaled_img"
)

In [112]:
import os
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from skimage import morphology, transform

In [113]:
def read_mask(filename, threshold):
    image_path = os.path.join(MASK_PATH, filename)

    image = Image.open(image_path)
    image = np.array(image.point(lambda p: 1 if p > threshold else 0))
    image = morphology.isotropic_opening(image, 9)

    if np.sum(image) > 5000:
        image = morphology.remove_small_objects(image, 512)
        _, connected = morphology.label(image, return_num=True)
        if connected == 2:
            return transform.resize(image, (64, 64))

In [146]:
def read_image(filename):
    image_path = os.path.join(IMAGE_PATH, filename)
    image = Image.open(image_path)
    return image.resize((64, 64))

In [150]:
out_masks = []

for path in os.listdir(MASK_PATH):
    mask = read_mask(path, 3)
    if mask is not None:
        out_masks.append(mask)
        mask_out = Image.fromarray(np.uint8(mask))
        mask_out.save(os.path.join(BINARY_MASK_OUT_PATH, path), "PNG")

        image = read_image(path)
        image.save(os.path.join(RESCALED_IMAGE_OUT_PATH, path), "PNG")